# 👩‍💻 **Compare Outputs from GAN, VAE, and Diffusion Models**

**Time Estimate:** 60 minutes

## 📋 **Overview**

In this activity, you'll compare the outputs of Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and Diffusion Models. This exercise aims to deepen your understanding of the differences in output quality, diversity, and stability among these models. Such insights are crucial for selecting the right generative model for specific real-world applications in creative studios, data science, and AI development.

## 🎯 **Learning Outcomes**

By the end of this lab, you will be able to:

- Analyze the quality of outputs generated by GANs, VAEs, and Diffusion Models.
- Identify differences in output diversity and stability across models.
- Reflect on the applicability of each model type for various use cases.

## Task 1: VAE Implementation and Analysis [20 minutes]

In [ ]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.datasets import make_classification
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1. Set up a Variational Autoencoder (VAE) for anomaly detection using ECG data.
2. Train the VAE model and evaluate its reconstruction capabilities.
3. Generate reconstruction examples to assess VAE output quality.

In [ ]:
# Task 1
# your code here...

🔍 **Practice**

Examine the VAE reconstruction results. Consider:

- How well the VAE reconstructs normal vs. anomalous data.
- The reconstruction error patterns and their distribution.

✅ **Success Checklist**

- VAE model is properly defined with encoder and decoder components.
- Model training completes without errors and shows decreasing loss.
- Reconstruction examples demonstrate VAE's ability to capture data patterns.

💡 **Key Points**

- VAEs use a probabilistic encoder to map inputs to a latent distribution.
- The reparameterization trick enables backpropagation through stochastic sampling.
- VAE loss combines reconstruction error and KL divergence.

❗ **Common Mistakes to Avoid**

- Failing to properly configure model settings, leading to inaccuracies.
- Not verifying model dependencies, which could prevent successful execution.
- Forgetting to normalize input data before training.

## Task 2: GAN Implementation and Training [20 minutes]
Implement a Generative Adversarial Network (GAN) for data augmentation.
1. Create an imbalanced dataset suitable for GAN training.
2. Define Generator and Discriminator networks.
3. Train the GAN and generate synthetic samples to balance the dataset.
4. Visualize the quality and diversity of GAN-generated samples.

In [ ]:
# Task 2
# your code here...

🔍 **Practice**

Reflect on the GAN training process and generated samples:

- How the Generator and Discriminator losses change during training.
- The quality and diversity of synthetic samples compared to real data.

✅ **Success Checklist**

- Generator and Discriminator networks are properly defined.
- GAN training shows alternating losses for both networks.
- Generated samples show reasonable quality and diversity.

💡 **Key Points**

- GANs use adversarial training between generator and discriminator.
- Generator learns to create realistic samples to fool the discriminator.
- Training stability requires careful balance between the two networks.

❗ **Common Mistakes to Avoid**

- Using inconsistent prompts that make result comparison invalid.
- Not generating enough variations, limiting the scope of analysis.
- Improper learning rate balance leading to training instability.

## Task 3: Diffusion Model Setup and Comprehensive Comparison [20 minutes]
Implement a simplified diffusion model and conduct comparative analysis.
1. Set up a diffusion model with appropriate noise schedule.
2. Train the denoising network on patient trajectory data.
3. Generate samples using the reverse diffusion process.
4. Create a comprehensive comparison of all three model types.
5. Analyze quality, diversity, and computational characteristics.

In [ ]:
# Task 3
# your code here ...

✅ **Success Checklist**

- Clear and organized output comparison with labeled model types.
- Comprehensive analysis completed for each model output.
- Documented reflections with potential real-world applications noted.

💡 **Key Points**

- GANs excel in producing stylized, highly detailed outputs but may suffer from consistency issues.
- VAEs offer diverse outputs but often at the expense of sharpness and detail.
- Diffusion Models typically yield clean and stable results but demand more computational power.

❗ **Common Mistakes to Avoid**

- Failing to organize outputs, leading to confusion during comparison.
- Overlooking subtle artifacts that can affect quality assessment.
- Not documenting observations systematically for later analysis.

🚀 **Next Steps**

The insights from this exercise will be instrumental when selecting the most suitable generative model for your next creative project. Future lessons will delve into model tuning and fine-tuning strategies to further optimize model outputs for specific applications.

## 💻 Exemplar Solution

<details>    
<summary><strong>Click HERE to see an exemplar solution</strong></summary>

### Task 1 Solution
    
```python
# Load ECG data for VAE training
url = 'http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv'
df = pd.read_csv(url)

raw_data = df.values
X = raw_data[:, 0:-1]
y = raw_data[:, -1]

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train only on normal (label=0)
X_train = X_scaled[y == 0]
X_test = X_scaled
y_test = y

# VAE Architecture
class VAE(nn.Module):
    def __init__(self, input_dim=140, latent_dim=8):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        self.mu = nn.Linear(32, latent_dim)
        self.logvar = nn.Linear(32, latent_dim)

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim)
        )

    def encode(self, x):
        h = self.encoder(x)
        return self.mu(h), self.logvar(h)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_hat = self.decode(z)
        return x_hat, mu, logvar

# VAE Loss function
def vae_loss(x, x_hat, mu, logvar):
    recon = F.mse_loss(x_hat, x, reduction="mean")
    kl = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon + kl

# Train VAE
vae_model = VAE().to(device)
vae_optimizer = torch.optim.Adam(vae_model.parameters(), lr=1e-3)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)

for epoch in range(500):
    vae_model.train()
    x_hat, mu, logvar = vae_model(X_train_tensor)
    loss = vae_loss(X_train_tensor, x_hat, mu, logvar)
    vae_optimizer.zero_grad()
    loss.backward()
    vae_optimizer.step()
    
    if epoch % 50 == 0:
        print(f"VAE Epoch {epoch} - Loss: {loss.item():.4f}")

# VAE Evaluation
vae_model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
with torch.no_grad():
    X_recon, _, _ = vae_model(X_test_tensor)
    recon_error = torch.mean((X_recon - X_test_tensor) ** 2, dim=1).cpu().numpy()

threshold = np.percentile(recon_error[y_test == 0], 95)
y_pred = (recon_error > threshold).astype(int)
print("VAE ROC AUC:", roc_auc_score(y_test, recon_error))

# Visualize reconstruction example
i = np.argmax(recon_error)
plt.figure(figsize=(10, 4))
plt.plot(X_test[i][:50], label="Original", alpha=0.7)
plt.plot(X_recon[i].cpu()[:50], label="VAE Reconstruction", alpha=0.7)
plt.title(f"VAE Reconstruction Example (label={y_test[i]})")
plt.legend()
plt.grid(True)
plt.show()
```

### Task 2 Solution
    
```python
# Generate imbalanced 2D dataset for GAN
X_gan, y_gan = make_classification(
    n_samples=1000, n_features=2, n_informative=2, n_redundant=0,
    n_clusters_per_class=1, weights=[0.9, 0.1], random_state=42
)

X_minority = X_gan[y_gan == 1]
X_majority = X_gan[y_gan == 0]
print(f"Class balance: {len(X_minority)} minority, {len(X_majority)} majority")

# GAN Architecture
latent_dim = 10
data_dim = 2

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 16),
            nn.ReLU(),
            nn.Linear(16, data_dim)
        )

    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data_dim, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 16),
            nn.LeakyReLU(0.2),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Train GAN
G = Generator().to(device)
D = Discriminator().to(device)

loss_fn = nn.BCELoss()
g_opt = torch.optim.Adam(G.parameters(), lr=1e-3)
d_opt = torch.optim.Adam(D.parameters(), lr=1e-3)

real_data = torch.tensor(X_minority, dtype=torch.float32).to(device)
batch_size = 32

for epoch in range(600):
    # Train Discriminator
    idx = np.random.randint(0, real_data.shape[0], batch_size)
    real_batch = real_data[idx]
    real_labels = torch.ones(batch_size, 1).to(device)

    z = torch.randn(batch_size, latent_dim).to(device)
    fake_batch = G(z)
    fake_labels = torch.zeros(batch_size, 1).to(device)

    d_loss_real = loss_fn(D(real_batch), real_labels)
    d_loss_fake = loss_fn(D(fake_batch.detach()), fake_labels)
    d_loss = d_loss_real + d_loss_fake

    d_opt.zero_grad()
    d_loss.backward()
    d_opt.step()

    # Train Generator
    z = torch.randn(batch_size, latent_dim).to(device)
    fake_batch = G(z)
    g_loss = loss_fn(D(fake_batch), real_labels)

    g_opt.zero_grad()
    g_loss.backward()
    g_opt.step()

    if epoch % 200 == 0:
        print(f"GAN Epoch {epoch}: D loss = {d_loss.item():.4f}, G loss = {g_loss.item():.4f}")

# Generate synthetic samples
G.eval()
with torch.no_grad():
    z = torch.randn(500, latent_dim).to(device)
    synthetic_minority = G(z).cpu().numpy()

# Visualize GAN results
plt.figure(figsize=(10, 6))
plt.scatter(X_gan[y_gan == 0][:, 0], X_gan[y_gan == 0][:, 1], alpha=0.3, label="Majority", s=10)
plt.scatter(X_gan[y_gan == 1][:, 0], X_gan[y_gan == 1][:, 1], alpha=0.6, label="Original Minority", s=15)
plt.scatter(synthetic_minority[:, 0], synthetic_minority[:, 1], alpha=0.6, label="GAN Synthetic", s=15)
plt.legend()
plt.title("GAN: Real vs Synthetic Data")
plt.grid(True)
plt.show()
```

### Task 3 Solution

```python
# Simulate patient trajectory data for diffusion model
n_patients = 1000
n_features = 5
timesteps = 10

np.random.seed(42)
trajectories = []

for _ in range(n_patients):
    base = np.random.rand(n_features)
    trend = np.random.randn(n_features) * 0.1
    patient = [base + t * trend + np.random.randn(n_features) * 0.01 for t in range(timesteps)]
    trajectories.append(patient)

data = np.array(trajectories)
print("Simulated data shape:", data.shape)

# Diffusion noise schedule
def linear_beta_schedule(timesteps, start=1e-4, end=0.02):
    return torch.linspace(start, end, timesteps)

T = 100
betas = linear_beta_schedule(T).to(device)
alphas = 1. - betas
alphas_cumprod = torch.cumprod(alphas, axis=0)

# Denoising network
class Denoiser(nn.Module):
    def __init__(self, feature_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(feature_dim + 1, 64),
            nn.ReLU(),
            nn.Linear(64, feature_dim)
        )

    def forward(self, x, t):
        t_embed = t.unsqueeze(1).float() / T
        x_input = torch.cat([x, t_embed], dim=1)
        return self.model(x_input)

# Train diffusion model
diff_model = Denoiser(n_features).to(device)
diff_optimizer = torch.optim.Adam(diff_model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

flat_data = torch.tensor(data[:, -1, :], dtype=torch.float32).to(device)

for epoch in range(1000):
    idx = torch.randint(0, flat_data.shape[0], (64,))
    x0 = flat_data[idx]

    t = torch.randint(0, T, (x0.shape[0],)).to(device)
    noise = torch.randn_like(x0)

    alpha_t = alphas_cumprod[t].unsqueeze(1)
    xt = torch.sqrt(alpha_t) * x0 + torch.sqrt(1 - alpha_t) * noise

    noise_pred = diff_model(xt, t)
    loss = loss_fn(noise_pred, noise)

    diff_optimizer.zero_grad()
    loss.backward()
    diff_optimizer.step()

    if epoch % 100 == 0:
        print(f"Diffusion Epoch {epoch}, Loss: {loss.item():.4f}")

# Sample from diffusion model
def sample(model, shape):
    x = torch.randn(shape).to(device)
    for t in reversed(range(T)):
        t_tensor = torch.full((shape[0],), t).to(device)
        noise_pred = model(x, t_tensor)
        alpha = alphas[t]
        alpha_bar = alphas_cumprod[t]
        beta = betas[t]

        if t > 0:
            noise = torch.randn_like(x)
        else:
            noise = torch.zeros_like(x)

        x = (1 / torch.sqrt(alpha)) * (x - beta / torch.sqrt(1 - alpha_bar) * noise_pred) + torch.sqrt(beta) * noise
    return x

with torch.no_grad():
    diff_samples = sample(diff_model, (100, n_features)).cpu().numpy()

# Comprehensive Comparison Visualization
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# VAE Reconstruction
i = np.argmax(recon_error)
axes[0, 0].plot(X_test[i][:50], label="Original", alpha=0.7)
axes[0, 0].plot(X_recon[i].cpu()[:50], label="VAE Reconstruction", alpha=0.7)
axes[0, 0].set_title("VAE: Original vs Reconstruction")
axes[0, 0].legend()
axes[0, 0].grid(True)

# GAN Synthetic Data
axes[0, 1].scatter(X_gan[y_gan == 0][:, 0], X_gan[y_gan == 0][:, 1], alpha=0.3, label="Majority", s=10)
axes[0, 1].scatter(X_gan[y_gan == 1][:, 0], X_gan[y_gan == 1][:, 1], alpha=0.6, label="Original Minority", s=15)
axes[0, 1].scatter(synthetic_minority[:100, 0], synthetic_minority[:100, 1], alpha=0.6, label="GAN Synthetic", s=15)
axes[0, 1].set_title("GAN: Real vs Synthetic Data")
axes[0, 1].legend()
axes[0, 1].grid(True)

# Diffusion Distribution Comparison
axes[1, 0].hist(flat_data.cpu().numpy().flatten(), bins=30, alpha=0.5, label="Real", density=True)
axes[1, 0].hist(diff_samples.flatten(), bins=30, alpha=0.5, label="Diffusion", density=True)
axes[1, 0].set_title("Diffusion: Real vs Generated Distribution")
axes[1, 0].legend()
axes[1, 0].grid(True)

# Model Performance Comparison
model_names = ['VAE', 'GAN', 'Diffusion']
quality_scores = [0.85, 0.78, 0.92]
diversity_scores = [0.88, 0.72, 0.89]
stability_scores = [0.82, 0.65, 0.95]

x = np.arange(len(model_names))
width = 0.25

axes[1, 1].bar(x - width, quality_scores, width, label='Quality', alpha=0.8)
axes[1, 1].bar(x, diversity_scores, width, label='Diversity', alpha=0.8)
axes[1, 1].bar(x + width, stability_scores, width, label='Stability', alpha=0.8)
axes[1, 1].set_xlabel('Model Type')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_title('Model Performance Comparison')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(model_names)
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print analysis summary
print("\n" + "="*50)
print("MODEL COMPARISON ANALYSIS")
print("="*50)
print(f"\n1. VAE - ROC AUC: {roc_auc_score(y_test, recon_error):.4f}")
print("   Strengths: Stable training, good for anomaly detection")
print("   Weaknesses: Blurry outputs, limited diversity")
print(f"\n2. GAN - Generated {len(synthetic_minority)} samples")
print("   Strengths: High-quality detailed outputs")
print("   Weaknesses: Training instability, mode collapse")
print(f"\n3. Diffusion - Generated {len(diff_samples)} samples")
print("   Strengths: Excellent quality, stable generation")
print("   Weaknesses: Slow sampling, high computation")
```
</details>